In [1]:
from datetime import datetime
from types import NoneType
from send2trash import send2trash
from model import all_dirs, ext, all_types, Base, Media, session, engine
from pathlib import Path
from pandas import DataFrame, read_sql
from itertools import chain
from exiftool import ExifToolHelper
from cv2 import VideoCapture, CAP_PROP_FRAME_COUNT, CAP_PROP_POS_FRAMES, COLOR_BGR2RGB, cvtColor
from PIL import Image
from xxhash import xxh64
from pillow_heif import register_heif_opener
from humanize import naturalsize

register_heif_opener()

In [2]:
df = read_sql("google_photos", con=engine)
df["path"] = df["path"].apply(Path) #type: ignore
df

,filename,path,type,size,size_norm,media,modify_date,exif_date,file_hash,media_hash,exif_data
0,IMG_20181014_000351_238.jpg,D:\RAW\Photos - Backup\2018\10\IMG_20181014_00...,jpg,329730,329.7 kB,image,2018-10-14 00:03:51.000000,2018-10-14 00:03:51,119662764340ee36,7d6f8d82ed4a9348,{'SourceFile': 'D:/RAW/Photos - Backup/2018/10...
1,IMG_20181015_084950.jpg,D:\RAW\Photos - Backup\2018\10\IMG_20181015_08...,jpg,329187,329.2 kB,image,2018-10-15 08:49:50.000000,2018-10-15 08:49:50,d95212cc7abc433a,af6fa4d58134cd24,"{'EXIF:ISO': 100, 'EXIF:Make': 'Xiaomi', 'EXIF..."
2,IMG_20181016_230510_539.jpg,D:\RAW\Photos - Backup\2018\10\IMG_20181016_23...,jpg,116787,116.8 kB,image,2018-10-16 23:05:10.000000,2018-10-16 23:05:10,15d32bfbfc6b3090,1ee4852bb2bcab12,{'SourceFile': 'D:/RAW/Photos - Backup/2018/10...
3,IMG_20181016_230955_569.jpg,D:\RAW\Photos - Backup\2018\10\IMG_20181016_23...,jpg,60888,60.9 kB,image,2018-10-16 23:09:55.000000,2018-10-16 23:09:55,12499748418116ec,43080e9522d609a4,{'SourceFile': 'D:/RAW/Photos - Backup/2018/10...
4,Screenshot 2025-03-26 222748.png,C:\Users\seshu\Pictures\Screenshots\Screenshot...,png,9317,9.3 kB,image,2025-03-26 22:27:48.678800,NaT,03c0b2c8fb55a9ab,447bd938e6963632,"{'PNG:Gamma': 2.2, 'PNG:Filter': 0, 'SourceFil..."
...,...,...,...,...,...,...,...,...,...,...,...
63040,IMG_20181013_100837.jpg,D:\RAW\Photos - Backup\2018\10\IMG_20181013_10...,jpg,742029,742.0 kB,image,2018-10-13 10:08:37.000000,2018-10-13 10:08:37,e821a0e7af036d6e,58d1e17aacd5011b,"{'EXIF:ISO': 125, 'EXIF:Make': 'Xiaomi', 'EXIF..."
63041,IMG_20181013_100906.jpg,D:\RAW\Photos - Backup\2018\10\IMG_20181013_10...,jpg,677554,677.6 kB,image,2018-10-13 10:09:06.000000,2018-10-13 10:09:06,ffbf01a99e16d7a0,ba94a8b79fe5d383,"{'EXIF:ISO': 100, 'EXIF:Make': 'Xiaomi', 'EXIF..."
63042,IMG_20181013_220355-01.jpg,D:\RAW\Photos - Backup\2018\10\IMG_20181013_22...,jpg,639868,639.9 kB,image,2018-10-13 22:03:55.000000,2018-10-13 22:03:55,56050d8d8f1ff979,151ceba55859e190,"{'EXIF:ISO': 640, 'EXIF:Make': 'Xiaomi', 'EXIF..."
63043,IMG_20181013_220355.jpg,D:\RAW\Photos - Backup\2018\10\IMG_20181013_22...,jpg,622490,622.5 kB,image,2018-10-13 22:03:55.000000,2018-10-13 22:03:55,6367a7348bf11619,61bd432587a2d5fd,"{'EXIF:ISO': 640, 'EXIF:Make': 'Xiaomi', 'EXIF..."


In [4]:
len(set(df["file_hash"]))

63045

In [3]:
from base64 import urlsafe_b64encode
from hashlib import sha256
from cryptography.fernet import Fernet


class Cipher:
    def __init__(self, password: str) -> None:
        self.password = password
        self.key = urlsafe_b64encode(sha256(password.encode()).digest())
        self.cipher = Fernet(self.key)
    
    def encrypt(self, message: str) -> str:
        return self.cipher.encrypt(message.encode()).decode()

    def decrypt(self, token: str) -> str:
        return self.cipher.decrypt(token.encode()).decode()

In [4]:
cipher = Cipher("19216821810128324630")

In [5]:
print(cipher.decrypt("gAAAAABn6ZaysJ_hEQfL7x63b6RP4kt9n8dF5ysMkGf5Pf88UQIHhH9rw3O1c_8lyJ7fPdqaFFehp3FicsC11XIA-lrj8FYf8v-eA9Ee76jvEi9RfgGtZUReMtfH6JE01eeNFKYF4YclXpAdWcQ2mQxX7kek0nNBukKD7WjGwBb8MHUlbCAwu2GDYBKnsguHSBKAPe14NImWSrNPswuX7n3A_Df7kZoUgYZBoPteUKXXb8I_D795f_w3uuVo5Zo_g3nDgOpJdhCCqtum56HXQf2VKAXFLlK5awJn-KV-jDkdZ17R_ltWm9UGONnbTJwMwAMqW82BP7V4JzrIVqdZaxutJo_MBt2NO_w9XZL_TLSHRhXzIWjuG98_Us1d4TTKKRPkpdGUYmbPoOAjZd2urAO-1mjv4ODwU3qRjPlNTFUPwzczsb8McIrGCDbGxvVd4cZ7mApi1LnHekGeyJUETS7Wy7l4jHJgF5iQSTw-uXBmrFWMNnjjXrJaaRQgt199XQLylmNSHcHXMyzDyYHY0HdShJrW5aceYZ3Dra5p6BBgHqLSA0EKH5DGxrXf_Ysm7Q7gNgp94oGR"))

SAVE YOUR BACKUP CODES
Keep these backup codes somewhere safe but accessible.

1. 8803 6922		 6. 4707 6904
2. 3425 4804		 7. 5031 1854
3. 8060 3709		 8. 6975 9006
4. 0612 6344		 9. 8961 7838
5. 6162 3397		10. 5936 8787

(chandu80743@gmail.com)

* You can only use each backup code once.
* Need more? Visit https://g.co/2sv
* These codes were generated on: Mar 31, 2025.
